# III. Deep Learning from scratch
## Recurrent Neural Networks

## 1 - Packages

In [1]:
import numpy as np
import time as tm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from dnn_utils import *

%matplotlib inline

## 2. Forward propogation for the basic Recurrent Neural Networks
### 2.1 - RNN cell
$$ a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a) $$
$$ \hat{y}^{\langle t \rangle} = softmax(W_{ya}a^{\langle t \rangle} + b_y)$$

In [9]:
def rnn_cell_forward(xt, a_prev, parameters):
    """
    Arguments:
        xt -- your input data at timestep "t", numpy array of shape (n_x, m)
        a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
        parameters -- python dictionary containing:
            Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
            Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
            Wya -- Weight matrix relating the hidden-state to the output, numpy of shape (n_y, n_a)
            ba -- Bias, numpy array of shape (n_a, 1)
            by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    
    Return:
        a_ next -- next hidden state, of shape (n_a, m)
        yt_pred -- prediction at timestep "t", numpy array of shape (a_next, a_prev, xt, parameters)
    """
    
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    a_next = np.tanh(np.add(np.add(np.dot(Wax, xt), np.dot(Waa, a_prev)), ba))
    yt_pred = softmax(np.add(np.dot(Wya, a_next), by))
    
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

### 2.2 - RNN forward pass

In [23]:
def rnn_forward(x, a0, parameters):
    """
    Arguments:
        x -- Input data for every time-step, of shape (n_x, m, T_x)
        a0 -- Initial hidden state, of shape (n_a, m)
        parameters -- python dictionary containing:
            Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
            Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
            Wya -- Weight matrix relating the hidden-state to the output, numpy of shape (n_y, n_a)
            ba -- Bias, numpy array of shape (n_a, 1)
            by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    
    Returns:
        a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
        y_pred -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
        caches -- tuple of values needed for the backward pass, contains (list of caches, x)
    """
    
    caches = []
    
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
    
    a_next = a0
    
    for t in range(T_x):
        a_next, yt_pred, cache = rnn_cell_forward(x[:,:,t], a_next, parameters)
        a[:,:,t] = a_next
        y_pred[:,:,t] = yt_pred
        caches.append(cache)
    
    caches = (caches, x)
    
    return a, y_pred, caches

## 3 - Long Short-Term Memory (LSTM) network
### About the gates
- Forget gate
$$ \Gamma_f^{\langle t \rangle} = \sigma(W_f[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_f)$$
- Update gate
$$ \Gamma_u^{\langle t \rangle} = \sigma(W_u[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_u)$$
- Updating the cell
$$ \hat{c}^{\langle t \rangle} = \tanh(W_c[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_c)$$
$$ c^{\langle t \rangle} = \Gamma_f^{\langle t \rangle} * c^{\langle t-1 \rangle} + \Gamma_u^{\langle t \rangle} * \hat{c}^{\langle t-1 \rangle}$$
- Output gate
$$ \Gamma_o^{\langle t \rangle} = \sigma(W_o[a^{\langle t-1 \rangle}, x^{\langle t \rangle}], b_o)$$
$$ a^{\langle t \rangle} = \Gamma_o^{\langle t \rangle} * \tanh(c^{\langle t \rangle})$$

### 2.1 - LSTM cell

In [2]:
def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    """
    Arguments:
        xt -- Input data at timestep "t", numpy array of shape (n_x, m)
        a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
        c_prev -- Memory state at timestep "t-1", numpy array of shape (n_a, m)
        parameters -- python dictionary containing:
            Wf -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
            bf -- Bias of the forget gate, numpy array of shape (n_a, 1)
            Wi -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
            bi -- Bias of the update gate, numpy array of shape (n_a, 1)
            Wc -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
            bc -- Bias of the first "tanh", numpy array of shape (n_a, 1)
            Wo -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
            bo -- Bias of the output gate, numpy array of shape (n_a, 1)
            Wy -- Weight matrix ralating the hidden-state to the output, numpy array of shape (n_y, n_a)
            by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    
    Returns:
        a_next -- next hidden state, of shape (n_a, m)
        c_next -- next memory state, of shape (n_a, m)
        yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
        cache -- tuple of values needed for the backward pass, contains (a_next, c_next, a_prev, c_prev, xt, parameters)
    """
    
    Wf = parameters["Wf"]
    bf = parameters["bf"]
    Wi = parameters["Wi"]
    bi = parameters["bi"]
    Wc = parameters["Wc"]
    bc = parameters["bc"]
    Wo = parameters["Wo"]
    bo = parameters["bo"]
    Wy = parameters["Wy"]
    by = parameters["by"]
    
    n_x, m = xt.shape
    n_y, n_a = Wy.shape
    
    concat = np.zeros((n_a+n_x, m))
    concat[:n_a,:] = a_prev
    concat[n_a:,:] = xt
    
    ft = sigmoid(np.add(np.dot(Wf, concat), bf))
    it = sigmoid(np.add(np.dot(Wi, concat), bi))
    cct = np.tanh(np.add(np.dot(Wc, concat), bc))
    c_next = np.add(np.multiply(ft, c_prev), np.multiply(it, cct))
    ot = sigmoid(np.add(np.dot(Wo, concat), bo))
    a_next = np.multiply(ot, np.tanh(c_next))
    
    yt_pred = softmax(np.add(np.dot(Wy, a_next), by))
    
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)
    
    return a_next, c_next, yt_pred, cache

### 2.2 - Forward pass for LSTM

In [4]:
def lstm_forward(x, a0, parameters):
    """
    Arguments:
        xt -- Input data at timestep "t", numpy array of shape (n_x, m)
        a0 -- Initial hidden state, numpy array of shape (n_a, m)
        parameters -- python dictionary containing:
            Wf -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
            bf -- Bias of the forget gate, numpy array of shape (n_a, 1)
            Wi -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
            bi -- Bias of the update gate, numpy array of shape (n_a, 1)
            Wc -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
            bc -- Bias of the first "tanh", numpy array of shape (n_a, 1)
            Wo -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
            bo -- Bias of the output gate, numpy array of shape (n_a, 1)
            Wy -- Weight matrix ralating the hidden-state to the output, numpy array of shape (n_y, n_a)
            by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    
    Returns:
        a -- Hidden satets for every time-step, of shape (n_a, m, T_x)
        yt_pred -- prediction for every time-step, numpy array of shape (n_y, m, T_x)
        cache -- tuple of values needed for the backward pass, contains (list of all the caches, x)
    """
    
    caches = []
    
    m_x, m, T_x = x.shape
    n_y, n_a = parameters['Wy'].shape
    
    a = np.zeros((n_a, m, T_x))
    c = np.zeros((n_a, m, T_x))
    y = np.zeros((n_y, m, T_x))
    
    a_next = a0
    c_next = np.zeros((n_a, m))
    
    for t in range(T_x):
        a_next, c_next, yt, cache = lstm_cell_forward(x[:,:,t], a_next, c_next, parameters)
        a[:,:,t] = a_next
        y[:,:,t] = yt
        c[:,:,t] = c_next
        caches.append(cache)
        
    caches = (caches, x)
    
    return a, y, c, caches

## 4 - Backpropagation in recurrent neural networks
### 4.1 - Basic RNN backward pass
$$ a^{\langle t \rangle} = \tanh(W_{ax}x^{\langle t \rangle}+W_{aa}a^{\langle t-1 \rangle}+b)$$
$$ \frac{\partial \tanh(x)}{\partial x} = 1 - \tanh(x)^2$$
$$ \frac{\partial a^{\langle t \rangle}}{\partial W_{ax}} = (1 - \tanh(W_{ax}x^{\langle t \rangle}+W_{aa}a^{\langle t-1 \rangle}+b)^2) · x^{\langle t \rangle T} $$
$$ \frac{\partial a^{\langle t \rangle}}{\partial W_{aa}} = (1 - \tanh(W_{ax}x^{\langle t \rangle}+W_{aa}a^{\langle t-1 \rangle}+b)^2) · a^{\langle t-1 \rangle T} $$
$$ \frac{\partial a^{\langle t \rangle}}{\partial b} = \sum\limits_{batch}(1 - \tanh(W_{ax}x^{\langle t \rangle}+W_{aa}a^{\langle t-1 \rangle}+b)^2) $$
$$ \frac{\partial a^{\langle t \rangle}}{\partial x^{\langle t \rangle}} = W_{ax}^T · (1 - \tanh(W_{ax}x^{\langle t \rangle}+W_{aa}a^{\langle t-1 \rangle}+b)^2) $$
$$ \frac{\partial a^{\langle t \rangle}}{\partial a^{\langle t-1 \rangle}} = W_{aa}^T · (1 - \tanh(W_{ax}x^{\langle t \rangle}+W_{aa}a^{\langle t-1 \rangle}+b)^2) $$

In [12]:
def rnn_cell_backward(da_next, cache):
    """
    Arguments:
        da_next -- Gradient of loss with respect to next hidden state
        cache -- python dictionary containing useful values (output of rnn_cell_forward())
    
    Returns:
        gradients -- python dictionary containing:
            dx -- Gradients of input data, of shape (n_x, m)
            da_prev -- Gradients of previous hidden state, of shape (n_a, n_x)
            dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
            dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
            dba -- Gradients of bias vector, of shape (n_a, 1)
    """
    
    (a_next, a_prev, xt, parameters) = cache
    
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    dtanh = np.multiply(np.subtract(1, np.power(a_next, 2)), da_next)
    
    dxt = np.dot(Wax.T, dtanh)
    dWax = np.dot(dtanh, xt.T)
    
    da_prev = np.dot(Waa.T, dtanh)
    dWaa = np.dot(dtanh, a_prev.T)
    
    dba = np.sum(dtanh, axis=-1, keepdims=True)
    
    gradients = {"dxt":dxt, "da_prev":da_prev, "dWax":dWax, "dWaa":dWaa, "dba":dba}
    
    return gradients

**Backwark pass through the RNN**

In [17]:
def rnn_backward(da, caches):
    """
    Arguments:
        da -- Upstream gradients of all hidden states, of shape (n_a, m, T_x)
        caches -- tuple containing information from the forward pass (rnn_forward)
    
    Returns:
        gradients -- python dictionary containing:
            dx -- Gradients of input data, of shape (n_x, m)
            da0 -- Gradients of initial hidden state, of shape (n_a, m)
            dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
            dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
            dba -- Gradients of bias vector, of shape (n_a, 1)
    """
    
    (caches, x) = caches
    (a1, a0, x1, parameters) = caches[0]
    
    n_a, m, T_x = da.shape
    n_x, m = x1.shape
    
    dx = np.zeros((n_x, m, T_x))
    dWax = np.zeros((n_a, n_x))
    dWaa = np.zeros((n_a, n_a))
    dba = np.zeros((n_a, 1))
    da0 = np.zeros((n_a, m))
    da_prevt = np.zeros((n_a, m))
    
    for t in reversed(range(T_x)):
        gradients = rnn_cell_backward(da[:,:,t]+da_prevt, caches[t])
        dxt, da_prevt, dWaxt, dWaat, dbat = gradients["dxt"], gradients["da_prev"], gradients["dWax"], gradients["dWaa"], gradients["dba"]
        dx[:,:,t] = dxt
        dWax += dWaxt
        dWaa += dWaat
        dba += dbat
    da0 = da_prevt
    
    gradients = {"dx":dx, "da0":da0, "dWax":dWax, "dWaa":dWaa, "dba":dba}
    
    return gradients

### 4.2 - LSTM backward pass

In [25]:
np.random.seed(1)
x = np.random.randn(3,10,4)
a0 = np.random.randn(5,10)
Wax = np.random.randn(5,3)
Waa = np.random.randn(5,5)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Wax":Wax, "Waa":Waa, "Wya":Wya, "ba":ba, "by":by}
a, y, caches = rnn_forward(x, a0, parameters)
da = np.random.randn(5,10,4)
gradients = rnn_backward(da, caches)
print("gradients[\"dx\"][1][2] = ", gradients["dx"][1][2])
print("gradients[\"dx\"].shape = ", gradients["dx"].shape)
print("gradients[\"da0\"][1][2] = ", gradients["da0"][2][3])
print("gradients[\"da0\"].shape = ", gradients["da0"].shape)
print("gradients[\"dWax\"][3][1] = ", gradients["dWax"][3][1])
print("gradients[\"dWax\"].shape = ", gradients["dWax"].shape)
print("gradients[\"dWaa\"][1][2] = ", gradients["dWaa"][1][2])
print("gradients[\"dWaa\"].shape = ", gradients["dWaa"].shape)
print("gradients[\"dba\"][4] = ", gradients["dba"][4])
print("gradients[\"dba\"].shape = ", gradients["dba"].shape)

gradients["dx"][1][2] =  [-2.07101689 -0.59255627  0.02466855  0.01483317]
gradients["dx"].shape =  (3, 10, 4)
gradients["da0"][1][2] =  -0.3149423751266499
gradients["da0"].shape =  (5, 10)
gradients["dWax"][3][1] =  11.264104496527779
gradients["dWax"].shape =  (5, 3)
gradients["dWaa"][1][2] =  2.3033331265798935
gradients["dWaa"].shape =  (5, 5)
gradients["dba"][4] =  [-0.74747722]
gradients["dba"].shape =  (5, 1)
